In [1]:
# Imports & Spark setup
import os
import shutil
import glob
import hdf5_getters
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType

from tools import setup_spark_config

sc, spark = setup_spark_config("Exploing Million Song Dataset")

In [2]:
# Inspect some sample data
h5 = hdf5_getters.open_h5_file_read('MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5')

# in byte format --> decode to string
print('Sample Artist: %s, Song: %s' % \
      (hdf5_getters.get_artist_name(h5).decode('UTF-8'), hdf5_getters.get_title(h5).decode('UTF-8')))

Sample Artist: Casual, Song: I Didn't Mean To


In [3]:
# Get all subdirs
def get_subdirs(basedir):
    subdirs = []
    for subdir in next(os.walk(basedir))[1]:
        subdirs.append(os.path.join(basedir, subdir))
    return subdirs

basedir = 'MillionSongSubset'
subdirs_rdd = sc.parallelize(get_subdirs(basedir))
subsubdirs_rdd = subdirs_rdd.map(lambda subdir: get_subdirs(subdir)).flatMap(lambda x: x)
subsubsubdirs_rdd = subsubdirs_rdd.map(lambda subsubdir: get_subdirs(subsubdir)).flatMap(lambda x: x)

In [4]:
print('%d dirs in dataset' % (subsubsubdirs_rdd.count()))
subsubsubdirs_rdd.take(5)

894 dirs in dataset


['MillionSongSubset/A/R/R',
 'MillionSongSubset/A/R/U',
 'MillionSongSubset/A/R/I',
 'MillionSongSubset/A/R/N',
 'MillionSongSubset/A/R/G']

In [5]:
# iterate & get all files (songs)
def count_and_get_files(basedir, ext='.h5'):
    # modified version of: https://labrosa.ee.columbia.edu/millionsong/pages/iterate-over-all-songs
    cnt = 0
    all_files = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for file in files:
            all_files.append(file)
        cnt += len(files)
    return cnt, all_files

file_names_rdd = subsubsubdirs_rdd.map(lambda subsubsubdir: count_and_get_files(subsubsubdir)[1]).flatMap(lambda x: x)

In [6]:
print('%d files in dataset' % (file_names_rdd.count()))
file_names_rdd.take(5)

10000 files in dataset


['MillionSongSubset/A/R/R/TRARRZU128F4253CA2.h5',
 'MillionSongSubset/A/R/R/TRARRJL128F92DED0E.h5',
 'MillionSongSubset/A/R/R/TRARRUZ128F9307C57.h5',
 'MillionSongSubset/A/R/R/TRARRWA128F42A0195.h5',
 'MillionSongSubset/A/R/R/TRARRPG12903CD1DE9.h5']

In [7]:
# put all files in rdd
files_rdd = file_names_rdd.map(lambda filename: hdf5_getters.open_h5_file_read(filename))

In [8]:
# get artist name for each song
def get_artist_name(file):
    return hdf5_getters.get_artist_name(file).decode('UTF-8')

artist_names_rdd = files_rdd.map(get_artist_name)

In [9]:
artist_names_rdd.take(5)

['Raphaël',
 'Julie Zenatti',
 'The Baltimore Consort',
 'I Hate Sally',
 'Orlando Pops Orchestra']

In [10]:
# get all artists and songs in format: (artist, song)
def get_artist_and_song(file):
    artist = hdf5_getters.get_artist_name(file).decode('UTF-8')
    song = hdf5_getters.get_title(file).decode('UTF-8')
    return artist, song

artist_song_rdd = files_rdd.map(lambda x: get_artist_and_song(x))

In [11]:
# group songs by their artist in format: (artist, [song1, song2, song3...])
grouped_artist_song_rdd = artist_song_rdd.groupByKey().mapValues(list)

In [12]:
n_unique_artists = grouped_artist_song_rdd.count()
print('%d unique artists in dataset' % (n_unique_artists))

for artist_songs in grouped_artist_song_rdd.take(5):
    print('\n%s has %d songs:' % (artist_songs[0], len(artist_songs[1])))
    songs = ''
    for song in artist_songs[1]:
        print('- %s' % (song))

4412 unique artists in dataset

Rauni Pekkala has 1 songs:
- Miesten tähden

Weezer has 4 songs:
- Dope Nose
- I Don't Want To Let You Go
- Don't Let Go
- Mykel And Carli

Tommy McLain has 1 songs:
- Let It Be Me

Tina Ann has 3 songs:
- Too Late (Orange Factory Extended Mix)
- I Do (Chris The Greek Remix)
- Rules of Attraction (Chris Cox)

Los Indios Tabajaras has 1 songs:
- Sueño Salvaje


In [13]:
# check if any artist has no songs -- shouldn't be possible
print('%d artists have no songs' % (grouped_artist_song_rdd.filter(lambda x: len(x[1]) < 1).count()))

0 artists have no songs


In [14]:
# get song data to use for analysis
def get_song_data(file):
    danceability = float(hdf5_getters.get_danceability(file))
    energy = float(hdf5_getters.get_energy(file))
    loudness = float(hdf5_getters.get_loudness(file))
    hotness = float(hdf5_getters.get_song_hotttnesss(file))
    year = int(hdf5_getters.get_year(file))
    return danceability, energy, loudness, hotness, year

songs_rdd = files_rdd.map(get_song_data)

In [15]:
songs_rdd.take(5)

[(0.0, 0.0, -9.636, 0.5479529419800353, 2008),
 (0.0, 0.0, -11.061, 0.47563846801023907, 2004),
 (0.0, 0.0, -24.14, nan, 0),
 (0.0, 0.0, -5.795, nan, 2007),
 (0.0, 0.0, -16.477, nan, 0)]

In [16]:
# move song data from RDD to DF & table view for optimization & Spark-SQL queries
fields = [StructField("danceability", FloatType()), \
          StructField("energy", FloatType()), \
          StructField("loudness", FloatType()), \
          StructField("hotness", FloatType()), \
          StructField("year", IntegerType())]

schema = StructType(fields)

songs_df = spark.createDataFrame(songs_rdd, schema)
songs_df.createOrReplaceTempView("songs")

In [17]:
songs_df.show()

+------------+------+--------+----------+----+
|danceability|energy|loudness|   hotness|year|
+------------+------+--------+----------+----+
|         0.0|   0.0|  -9.636|0.54795295|2008|
|         0.0|   0.0| -11.061|0.47563848|2004|
|         0.0|   0.0|  -24.14|       NaN|   0|
|         0.0|   0.0|  -5.795|       NaN|2007|
|         0.0|   0.0| -16.477|       NaN|   0|
|         0.0|   0.0| -12.474|0.44545454|   0|
|         0.0|   0.0|  -4.393|0.32773668|   0|
|         0.0|   0.0|   -5.05|       NaN|   0|
|         0.0|   0.0|  -4.264| 0.7883882|1982|
|         0.0|   0.0| -13.885|       NaN|1998|
|         0.0|   0.0|  -4.707|  0.681092|2004|
|         0.0|   0.0|  -4.523|0.40148672|2005|
|         0.0|   0.0|  -4.076| 0.6878737|2004|
|         0.0|   0.0| -19.293| 0.2669552|   0|
|         0.0|   0.0|  -3.312|0.35528553|2001|
|         0.0|   0.0|  -6.619|0.54352427|   0|
|         0.0|   0.0| -25.651|0.21508032|1982|
|         0.0|   0.0|  -6.052|0.87222904|2000|
|         0.0

In [18]:
# filter out songs with NaN values and no year
filtered_songs_df = spark.sql("SELECT * FROM songs WHERE \
                                  isNaN(danceability) = false AND \
                                  isNaN(energy) = false AND \
                                  isNaN(hotness) = false AND \
                                  isNaN(year) = false AND \
                                  year > 0")
filtered_songs_df.createOrReplaceTempView("songs")

In [19]:
filtered_songs_df.show()

+------------+------+--------+----------+----+
|danceability|energy|loudness|   hotness|year|
+------------+------+--------+----------+----+
|         0.0|   0.0|  -9.636|0.54795295|2008|
|         0.0|   0.0| -11.061|0.47563848|2004|
|         0.0|   0.0|  -4.264| 0.7883882|1982|
|         0.0|   0.0|  -4.707|  0.681092|2004|
|         0.0|   0.0|  -4.523|0.40148672|2005|
|         0.0|   0.0|  -4.076| 0.6878737|2004|
|         0.0|   0.0|  -3.312|0.35528553|2001|
|         0.0|   0.0| -25.651|0.21508032|1982|
|         0.0|   0.0|  -6.052|0.87222904|2000|
|         0.0|   0.0| -15.433| 0.5968407|1981|
|         0.0|   0.0|  -4.325| 0.6248335|2007|
|         0.0|   0.0|  -5.193|0.42744657|2008|
|         0.0|   0.0|  -6.712|       0.0|2004|
|         0.0|   0.0|   -4.13| 0.4871122|2007|
|         0.0|   0.0|  -7.687|0.28848165|1978|
|         0.0|   0.0|  -7.687| 0.5675917|1995|
|         0.0|   0.0|  -21.82|0.50403434|2000|
|         0.0|   0.0|  -5.548|    0.5764|2005|
|         0.0

In [20]:
# write songs to parquet (better than CSV)
dst_dir = 'parsed_songs'
if os.path.isdir(dst_dir):
    shutil.rmtree(dst_dir)
filtered_songs_df.write.parquet('parsed_songs')